In [1]:
import os
import re
import pickle as pkl
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import scorpyo as sp

from null_importance import get_null_importance


pd.set_option('max_rows', 500, 'max_columns', 200)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_jupyter  = os.path.join(path_results, 'jupyter')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


path_new_train = os.path.join(path_new_data, 'train_lightgbm_20221014.csv')
path_new_test  = os.path.join(path_new_data, 'test_lightgbm_20221014.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"

In [3]:
df_row_train = sp.read_data(path_train)
df_row_val  = sp.read_data(path_test)

df_row_train['url_sit'] = df_row_train['url'].map(lambda x: x.split('/')[0])
df_row_train['url_page'] = df_row_train['url'].map(lambda x: x.split('/')[1])

df_row_val['url_sit'] = df_row_val['url'].map(lambda x: x.split('/')[0])
df_row_val['url_page'] = df_row_val['url'].map(lambda x: x.split('/')[1])



df_train_info = df_row_train.copy().sort_values(by='op_datetime')

## 定义一次/一天/一月
1. op\_times\_groups 一次
2. op\_days 一天
3. op\_month 一月

In [4]:
df = pd.concat([df_row_train, df_row_val]).reset_index(drop=True)
df = df.sort_values(by='op_datetime')

# 认证日期时间
df['op_datetime'] = pd.to_datetime(df['op_datetime'])
# 将数据分为每一天
df['op_days'] = df['op_datetime'].map(lambda x: x.strftime('%Y-%m-%d'))

# 按客户进行统计这次认证和上次认证的时间差
df['op_second'] = df['op_datetime']
df['op_second1'] = df.groupby('device_num_transform')['op_second'].shift(1)
df['op_diff_second1'] = (df['op_second'] - df['op_second1']).map(lambda x: x.total_seconds())
df['op_diff_second1_log'] = df['op_diff_second1'].apply(np.log)
df['op_diff_second1_log_log'] = df['op_diff_second1'].apply(np.log).apply(np.log)

df['op_diff_second1_tmp'] = df['op_diff_second1']>10
# 客户第几次登录
df['op_times_groups'] = df.groupby('device_num_transform')['op_diff_second1_tmp'].apply(lambda x: x.cumsum())

# 系统层面的一段时间
df['system_op_second'] = df['op_datetime']
df['system_op_second1'] = df['system_op_second'].shift(1)
df['system_op_diff_second1'] = (df['system_op_second'] - df['system_op_second1']).map(lambda x: x.total_seconds())

df['system_op_diff_second1_log'] = df['system_op_diff_second1'].apply(np.log)
df['system_op_diff_second1_log_log'] = df['system_op_diff_second1'].apply(np.log).apply(np.log)
df['system_op_diff_second1_tmp'] = df['system_op_diff_second1']>400
# 客户第几次登录
df['system_op_times_groups'] = df['system_op_diff_second1_tmp'].cumsum()

df = df.drop(columns=['op_second','op_second1','op_diff_second1_tmp', 
                      'system_op_second', 'system_op_second1', 'system_op_diff_second1_tmp'])

## 环境特征

In [5]:

time_feats = ['system_op_times_groups', 'op_days', 'op_month']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url', 'url_page','url_sit']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()

for i in time_feats:
    i_tmp = df.groupby([i])
    # 系统往前看，处理了多少事
    df['system_{}_cumsum'.format(i)] = i_tmp['helper'].cumsum()
    # 系统往前看，处理了多少坏事
    df['system_{}_error_code_cumsum'.format(i)] = i_tmp['http_status_code_helper'].cumsum()
    for method in ['mean', 'std', "prod"]:
        df['system_{}_op_diff_second1_{}'.format(i, method)] = i_tmp['op_diff_second1'].transform(method)
    for j in cate_feats:
        index_set = set(df.groupby([i, j],as_index=False).first()['sampler_index_helper'].values)
        df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
        j_tmp = df.groupby([i, j])
        # 系统往前看，不同维度的处理了多少情况
        df['system_{}_{}_cumunique'.format(i, j)] = j_tmp['tmp_helper'].cumsum()
        
        if i=='system_op_times_groups':
            for method in ['mean', 'std']:
                df['system_{}_{}_op_diff_second1_{}'.format(i,j, method)] = j_tmp['op_diff_second1'].transform(method)

        if j not in ['ip_transform', 'user_name', 'device_num_transform']:
            for k in df[j].unique():
                tmp = df[df[j]==k].groupby([i])

                # 系统往前看，不同维度不同情况分别处理了多少次
                df['system_{}_{}_{}_cumsum'.format(i,j,k)] = tmp['helper'].cumsum()
                # 系统往前看，不同维度不同情况error_code分别处理了多少次
                df['system_{}_{}_{}_error_code_cumsum'.format(i,j,k)] = tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

## 对象层面

In [6]:

time_feats = ['op_times_groups', 'op_days', 'op_month']

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

cate_feats = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url', 'url_page','url_sit']

# 客户+时间+按时间cumsum/cumunique
# 累计量统计
df['helper'] = 1
# 是否为异常状态码
df['http_status_code_helper'] = df['http_status_code'].map(lambda x: 1 if x in [400, 500, 502, 404] else 0) 
# 给样本编号
df['sampler_index_helper'] = df['helper'].cumsum()


for i in time_feats:
    for j in object_feats:
        j_tmp = df.groupby([i,j])
        df['{}_{}_cumsum'.format(i,j)] = j_tmp['helper'].cumsum()
        df['{}_{}_error_code_cumsum'.format(i,j)] = j_tmp['http_status_code_helper'].cumsum()
        
        if i=='op_times_groups':
            for method in ['mean', 'std']:
                df['{}_{}_op_diff_second1_{}'.format(i, j, method)] = j_tmp['op_diff_second1'].transform(method)

        for k in cate_feats:
            if k == j: continue
            index_set = set(df.groupby([i,j,k], as_index=False).first()['sampler_index_helper'].values)            
            df['tmp_helper'] = df['sampler_index_helper'].map(lambda x: 1 if x in index_set else 0)
            k_tmp = df.groupby([i,j,k])
            df['{}_{}_{}_cumunique'.format(i,j,k)] = k_tmp['tmp_helper'].cumsum()
            for method in ['mean', 'std', "prod"]:
                df['{}_{}_{}_op_diff_second1_{}'.format(i, j, k, method)] = k_tmp['op_diff_second1'].transform(method)

            if k not in ['ip_transform', 'user_name', 'device_num_transform']:
                for v in df[k].unique():
                    v_tmp = df[df[k]==v].groupby([i,j])
                    df['{}_{}_{}_{}_cumsum'.format(i,j,k,v)] = v_tmp['helper'].cumsum()
                    df['{}_{}_{}_{}_error_code_cumsum'.format(i,j,k,v)] = v_tmp['http_status_code_helper'].cumsum()

remove_cols = [x for x in df.columns if x[-6:]=='helper']

df = df.drop(columns=remove_cols)

## 其它操作

In [7]:
# 几点钟
df['hour'] = df['op_datetime'].dt.hour
df['hour_sin'] = np.sin(df['hour']/24*2*np.pi)
df['hour_cos'] = np.cos(df['hour']/24*2*np.pi)

# 周几
df['dayofweek'] = df['op_datetime'].dt.dayofweek
df['dayofweek_sin'] = np.sin(df['dayofweek']/7*2*np.pi)
df['dayofweek_cos'] = np.cos(df['dayofweek']/7*2*np.pi)

# 一个月的第几天
df['day'] = df['op_datetime'].dt.day
df['day_sin'] = np.sin(df['day']/31*2*np.pi)
df['day_cos'] = np.cos(df['day']/31*2*np.pi)

In [8]:
with open(os.path.join(path_new_data, 'times_data.pkl'), 'wb') as f:
    pkl.dump(df, f)

In [4]:
with open(os.path.join(path_new_data, 'no_times_data.pkl'), 'rb') as f:
    df = pkl.load(f)

## 特征筛选

In [5]:
# 删除时间及类别型变量过多的特征
remove_col = ['op_datetime', 'op_month', 'user_name', 
              'op_times_groups', 'system_op_times_groups', 'op_days', 'op_month',
              'ip_transform', 'device_num_transform', 'op_days', 'ts', 'ts1', 'ts2','ts3']

remove_col = [x for x in df.columns if x in remove_col]
df = df.drop(columns=remove_col)
df.head()

,id,department,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_diff_second1_log,op_diff_second1_log_log,system_op_diff_second1,system_op_diff_second1_log,system_op_diff_second1_log_log,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_safari_cumsum,system_system_op_times_groups_browser_safari_error_code_cumsum,system_system_op_times_groups_browser_firefox_cumsum,system_system_op_times_groups_browser_firefox_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_browser_ie_cumsum,system_system_op_times_groups_browser_ie_error_code_cumsum,system_system_op_times_groups_os_type_cumunique,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_type_macos_cumsum,system_system_op_times_groups_os_type_macos_error_code_cumsum,system_system_op_times_groups_os_version_cumunique,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_os_version_win11_cumsum,system_system_op_times_groups_os_version_win11_error_code_cumsum,system_system_op_times_groups_ip_type_cumunique,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_

In [6]:
df_row_train = df[df[y_label].notna()].reset_index(drop=True)
df_row_val = df[df[y_label].isna()].reset_index(drop=True)

df_train, df_test, convert_cols = sp.transform_data_detail(df_row_train, df_row_val, y_label, excel_path=path_output_report)
df_train.head()

sheet05.可能为数值类型的object类型数据统计在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
sheet06.数据预处理在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


,id,department,browser_version,browser,os_version,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_diff_second1_log,op_diff_second1_log_log,system_op_diff_second1,system_op_diff_second1_log,system_op_diff_second1_log_log,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_cumsum,system_system_op_times_groups_http_status_code_200_cumsum,system_system_op_times_groups_op_city_深圳_cumsum,system_system_op_times_groups_op_city_深圳_error_code_cumsum,system_system_op_times_groups_op_city_成都_cumsum,system_system_op_times_groups_op_city_成都_error_code_cumsum,system_system_op_times_groups_op_city_杭州_cumsum,system_system_op_times_groups_op_city_杭州_error_code_cumsum,system_system_op_times_groups_op_city_北京_cumsum,system_system_op_times_groups_op_city_北京_error_code_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_error_code_cumsum,system_system_op_times_groups_url_xxx.com/github_cumsum,system_system_op_times_groups_url_xxx.com/github_error_code_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_error_code_cumsum,system_system_op_times_groups_url_page_github_cumsum,system_system_op_times_groups_url_page_github_error_code_cumsum,system_system_op_times_groups_url_page_download_cumsum,system_system_op_times_groups_url_page_download_error_code_cumsum,system_system_op_times_groups_url_sit_xxx.com_cumsum,system_system_op_times_groups_url_sit_xxx.com_error_code_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_error_code_cumsum,system_op_days_cumsum,system_op_days_error_code_cumsum,system_op_days_department_rd_cumsum,system_op_days_department_rd_error_code_cumsum,system_op_days_department_sales_cumsum,system_op_days_department_sales_error_code_cumsum,system_op_days_browser_version_edge_93_cumsum,system_op_days_browser_version_edge_93_error_code_cumsum,system_op_days_browser_version_chrome_90_cumsum,system_op_days_browser_version_chrome_90_error_code_cumsum,system_op_days_browser_chrome_cumsum,system_op_days_browser_chrome_error_code_cumsum,system_op_days_browser_edge_cumsum,system_op_days_browser_edge_error_code_cumsum,system_op_days_os_type_win_cumsum,system_op_days_os_type_win_error_code_cumsum,system_op_days_os_version_win10_cumsum,system_op_days_os_version_win10_error_code_cumsum,system_op_days_os_version_win7_cumsum,system_op_days_os_version_win7_error_code_cumsum,system_op_days_ip_type_内网_cumsum,system_op_days_ip_type_内网_error_code_cumsum,system_op

In [7]:
# df_train = df[df[y_label].notna()].reset_index(drop=True)
# df_test = df[df[y_label].isna()].reset_index(drop=True)

df_train_extend = df_train[[y_label,'id']]
df_test_extend = df_test[[y_label,'id']]

In [8]:
# df_train = df_train.fillna(-999)
# df_test = df_test.fillna(-999)

### null importance特征筛选

In [9]:
feats, categorical_feats = get_null_importance(df_train.drop(columns=[y_label,'id']).copy(),
                                               df_train[y_label].copy(), 
                                               thresholds=15)

In [10]:
feats = [i for i in feats if i not in categorical_feats]

### 相关系数筛选

In [11]:
corr_matrix, corr_drop_var, df_train = sp.feature_selection_corr(df_train[feats+[y_label]], y_label)
feats = df_train.columns.drop([y_label])
print(corr_drop_var)

['op_times_groups_ip_transform_url_wpsdoc.xxx.com/download_cumsum'
 'op_times_groups_ip_transform_url_page_download_cumsum'
 'op_times_groups_ip_transform_log_system_transform_dwS3cdn15GK4_cumsum'
 'op_month_user_name_os_version_win7_cumsum'
 'op_days_ip_transform_url_page_download_cumsum'
 'op_days_user_name_log_system_transform_dwS3cdn15GK4_cumsum'
 'op_days_ip_transform_log_system_transform_dwS3cdn15GK4_cumsum'
 'op_days_ip_transform_url_sit_wpsdoc.xxx.com_cumsum'
 'op_month_ip_transform_os_version_win7_cumsum'
 'op_month_user_name_cumsum' 'op_month_user_name_ip_type_内网_cumsum'
 'op_month_device_num_transform_ip_type_内网_cumsum'
 'op_month_ip_transform_ip_type_内网_cumsum'
 'op_days_ip_transform_ip_type_内网_error_code_cumsum'
 'op_month_user_name_op_city_北京_cumsum'
 'op_days_user_name_log_system_transform_dwS3cdn15GK4_error_code_cumsum'
 'op_month_device_num_transform_os_type_win_cumsum'
 'op_month_ip_transform_os_type_win_cumsum'
 'op_times_groups_ip_transform_url_wpsdoc.xxx.com/downlo

In [12]:

object_feats = ['ip_transform', 'user_name', 'device_num_transform']

df_train = pd.concat([df_train_info[object_feats], df_train[feats], df_train_extend], axis=1)
df_test = pd.concat([df_test[feats], df_test_extend], axis=1)

## modeling

In [13]:
feats = df_train.columns.drop(['id', y_label]+object_feats)
#feats = feats.drop(categorical_feats)
#feats 

In [14]:
feats = feats_importance.sort_values('importance', ascending=False)[:50]['name'].values

NameError: name 'feats_importance' is not defined

In [15]:

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

In [16]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

In [17]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label],
                           # categorical_feature=categorical_feats
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label],
                          #categorical_feature=categorical_feats
                         )
        model = lgb.train(params, train, valid_sets=[val], 
                          num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[1]	valid_0's auc: 0.934617
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.938108
[3]	valid_0's auc: 0.938197
[4]	valid_0's auc: 0.942813
[5]	valid_0's auc: 0.936592
[6]	valid_0's auc: 0.939876
[7]	valid_0's auc: 0.941801
[8]	valid_0's auc: 0.941917
[9]	valid_0's auc: 0.941456
[10]	valid_0's auc: 0.941434
[11]	valid_0's auc: 0.941491
[12]	valid_0's auc: 0.941306
[13]	valid_0's auc: 0.941261
[14]	valid_0's auc: 0.942017
[15]	valid_0's auc: 0.941964
[16]	valid_0's auc: 0.941937
[17]	valid_0's auc: 0.941975
[18]	valid_0's auc: 0.942007
[19]	valid_0's auc: 0.94202
[20]	valid_0's auc: 0.941566
[21]	valid_0's auc: 0.942013
[22]	valid_0's auc: 0.942035
[23]	valid_0's auc: 0.940451
[24]	valid_0's auc: 0.940456
[25]	valid_0's auc: 0.940453
[26]	valid_0's auc: 0.940422
[27]	valid_0's auc: 0.940426
[28]	valid_0's auc: 0.941017
[29]	valid_0's auc: 0.940988
[30]	valid_0's auc: 0.940939
[31]	valid_0's auc: 0.940989
[32]	valid_0's auc: 0.940974
[33]	v

[69]	valid_0's auc: 0.937524
[70]	valid_0's auc: 0.938042
[71]	valid_0's auc: 0.938575
[72]	valid_0's auc: 0.939177
[73]	valid_0's auc: 0.939188
[74]	valid_0's auc: 0.939112
[75]	valid_0's auc: 0.9389
[76]	valid_0's auc: 0.938685
[77]	valid_0's auc: 0.938675
[78]	valid_0's auc: 0.93856
[79]	valid_0's auc: 0.938622
[80]	valid_0's auc: 0.938618
[81]	valid_0's auc: 0.938645
[82]	valid_0's auc: 0.938702
[83]	valid_0's auc: 0.938773
[84]	valid_0's auc: 0.938446
[85]	valid_0's auc: 0.938325
[86]	valid_0's auc: 0.938392
[87]	valid_0's auc: 0.938444
[88]	valid_0's auc: 0.938672
[89]	valid_0's auc: 0.938365
[90]	valid_0's auc: 0.938148
[91]	valid_0's auc: 0.938362
[92]	valid_0's auc: 0.938356
[93]	valid_0's auc: 0.938334
[94]	valid_0's auc: 0.938782
[95]	valid_0's auc: 0.938726
[96]	valid_0's auc: 0.938662
[97]	valid_0's auc: 0.938741
[98]	valid_0's auc: 0.938762
[99]	valid_0's auc: 0.938741
[100]	valid_0's auc: 0.938887
[101]	valid_0's auc: 0.938926
[102]	valid_0's auc: 0.938709
[103]	valid_0'

[90]	valid_0's auc: 0.935479
[91]	valid_0's auc: 0.935637
[92]	valid_0's auc: 0.935681
[93]	valid_0's auc: 0.935935
[94]	valid_0's auc: 0.935645
[95]	valid_0's auc: 0.93594
[96]	valid_0's auc: 0.935805
[97]	valid_0's auc: 0.93567
[98]	valid_0's auc: 0.935683
[99]	valid_0's auc: 0.935681
[100]	valid_0's auc: 0.93571
[101]	valid_0's auc: 0.935786
[102]	valid_0's auc: 0.935719
[103]	valid_0's auc: 0.9357
[104]	valid_0's auc: 0.935502
[105]	valid_0's auc: 0.935447
[106]	valid_0's auc: 0.935403
[107]	valid_0's auc: 0.935225
[108]	valid_0's auc: 0.935197
[109]	valid_0's auc: 0.935332
[110]	valid_0's auc: 0.935409
[111]	valid_0's auc: 0.935301
[112]	valid_0's auc: 0.935437
[113]	valid_0's auc: 0.935321
[114]	valid_0's auc: 0.935226
[115]	valid_0's auc: 0.935424
[116]	valid_0's auc: 0.935485
[117]	valid_0's auc: 0.935295
[118]	valid_0's auc: 0.935407
[119]	valid_0's auc: 0.935424
[120]	valid_0's auc: 0.935687
[121]	valid_0's auc: 0.935798
[122]	valid_0's auc: 0.935837
[123]	valid_0's auc: 0.93

,name,importance
4,system_system_op_times_groups_http_status_code...,78589.582168
23,op_month_ip_transform_http_status_code_200_cumsum,20210.851813
18,op_times_groups_device_num_transform_http_stat...,16020.880544
0,op_diff_second1,13790.921750
9,system_op_days_http_status_code_200_cumsum,12890.841836
25,hour_cos,9626.377788
1,system_op_diff_second1_log_log,6587.890471
17,op_times_groups_user_name_http_status_code_200...,3539.848536
24,op_month_ip_transform_url_wpsdoc.xxx.com/downl...,2420.872291
19,op_days_ip_transform_cumsum,1994.404502


0.9415886809918348

0.0022016056250611857

In [18]:
feats_importance.sort_values('importance', ascending=False).shape

(30, 2)

In [25]:
tmp

array(['day_cos', 'day_sin', 'dayofweek', 'dayofweek_sin', 'hour_cos',
       'op_days_ip_transform_cumsum',
       'op_days_ip_transform_os_type_win_error_code_cumsum',
       'op_days_ip_transform_url_sit_xxx.com_cumsum',
       'op_days_user_name_os_version_win10_error_code_cumsum',
       'op_diff_second1',
       'op_month_ip_transform_http_status_code_200_cumsum',
       'op_month_ip_transform_url_wpsdoc.xxx.com/download_cumsum',
       'op_times_groups_device_num_transform_http_status_code_200_cumsum',
       'op_times_groups_user_name_browser_chrome_error_code_cumsum',
       'op_times_groups_user_name_browser_version_edge_93_cumsum',
       'op_times_groups_user_name_error_code_cumsum',
       'op_times_groups_user_name_http_status_code_200_cumsum',
       'op_times_groups_user_name_os_type_win_error_code_cumsum',
       'system_op_days_http_status_code_200_cumsum',
       'system_op_days_op_city_北京_error_code_cumsum',
       'system_op_days_op_city_成都_error_code_cumsum',
    

In [19]:
tmp = feats_importance.sort_values('importance', ascending=False)['name'].values
tmp.sort()
for i in tmp:
    print(i)

day_cos
day_sin
dayofweek
dayofweek_sin
hour_cos
op_days_ip_transform_cumsum
op_days_ip_transform_os_type_win_error_code_cumsum
op_days_ip_transform_url_sit_xxx.com_cumsum
op_days_user_name_os_version_win10_error_code_cumsum
op_diff_second1
op_month_ip_transform_http_status_code_200_cumsum
op_month_ip_transform_url_wpsdoc.xxx.com/download_cumsum
op_times_groups_device_num_transform_http_status_code_200_cumsum
op_times_groups_user_name_browser_chrome_error_code_cumsum
op_times_groups_user_name_browser_version_edge_93_cumsum
op_times_groups_user_name_error_code_cumsum
op_times_groups_user_name_http_status_code_200_cumsum
op_times_groups_user_name_os_type_win_error_code_cumsum
system_op_days_http_status_code_200_cumsum
system_op_days_op_city_北京_error_code_cumsum
system_op_days_op_city_成都_error_code_cumsum
system_op_days_op_city_深圳_error_code_cumsum
system_op_days_os_version_win7_error_code_cumsum
system_op_diff_second1_log_log
system_system_op_times_groups_browser_version_edge_93_error_co

In [24]:
df_train.columns

Index(['ip_transform', 'user_name', 'device_num_transform', 'op_diff_second1',
       'system_op_diff_second1_log_log',
       'system_system_op_times_groups_browser_version_edge_93_error_code_cumsum',
       'system_system_op_times_groups_os_version_win7_error_code_cumsum',
       'system_system_op_times_groups_http_status_code_200_cumsum',
       'system_system_op_times_groups_op_city_深圳_error_code_cumsum',
       'system_system_op_times_groups_op_city_杭州_error_code_cumsum',
       'system_system_op_times_groups_op_city_北京_error_code_cumsum',
       'system_op_days_os_version_win7_error_code_cumsum',
       'system_op_days_http_status_code_200_cumsum',
       'system_op_days_op_city_深圳_error_code_cumsum',
       'system_op_days_op_city_成都_error_code_cumsum',
       'system_op_days_op_city_北京_error_code_cumsum',
       'op_times_groups_user_name_error_code_cumsum',
       'op_times_groups_user_name_browser_version_edge_93_cumsum',
       'op_times_groups_user_name_browser_chrome_error

In [23]:
df_train['op_times_groups_device_num_transform_wpsdoc.xxx.com/download_cumsum']

KeyError: 'op_times_groups_device_num_transform_wpsdoc.xxx.com/download_cumsum'

In [20]:
df_train_info['oof']=df_train['oof']

df_train_info[abs(df_train_info['is_risk']-df_train_info['oof'])>0.7]['device_num_transform'].value_counts()

HL3vrsyu1H3Z    79
0dV6LzVsv7pW    76
Rfv57YyO3vny    73
5DmlITfRNR36    71
8hjqBqrNDPkE    70
                ..
6DDzOi2BV383     1
9nRsk1CCOdLt     1
3uBfpGYbfD3Q     1
7D0GPTvDM4Fn     1
PQZXB2FgV30B     1
Name: device_num_transform, Length: 727, dtype: int64

In [21]:
df_train_info[df_train_info['device_num_transform']=='0dV6LzVsv7pW']

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,url_sit,url_page,oof
581,581,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-07 18:21:48,内网,200,杭州,nHrKgKdJ1Mzt,xxx.com/github,2022-01,0,xxx.com,github,0.317583
44887,44887,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-10 23:31:25,内网,200,杭州,fwM6KZKjrzjm,xxx.com/oa,2022-01,1,xxx.com,oa,0.053633
14489,14489,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:00,内网,200,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/kdocs,2022-01,0,wpsdoc.xxx.com,kdocs,0.034824
36352,36352,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:01,内网,200,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.053481
36353,36353,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:04,内网,200,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.035647
36354,36354,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:06,内网,200,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.063371
36355,36355,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:07,内网,200,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.035754
36356,36356,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:10,内网,200,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,0,wpsdoc.xxx.com,download,0.062927
36357,36357,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:12,内网,400,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,1,wpsdoc.xxx.com,download,0.053422
36358,36358,lufan2545,rd,5KbVyNsBf,0dV6LzVsv7pW,edge_93,edge,win,win10,2022-01-11 10:33:14,内网,400,杭州,dwS3cdn15GK4,wpsdoc.xxx.com/download,2022-01,1,wpsdoc.xxx.com,download,0.064526
